In [1]:
#####################################################################
#########   MAIN CODE FOR  CSV FILESS TO DOWNLOAD    #########
#####################################################################

import time
import os
import csv
from datetime import datetime as dt_datetime, timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import glob

# Define the path to the folder containing the CSV files
folder_path = r'C:\Users\omkar\Downloads'

def delete_csv_files():
    """Delete only CSV files in the folder."""
    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    for file in csv_files:
        try:
            os.remove(file)
            print(f"Deleted: {file}")
        except Exception as e:
            print(f"Error deleting {file}: {e}")

def download_csv(chrome_driver_path, login_url, username, password, csv_file_name):
    # Initialize Chrome WebDriver
    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)
    
    try:
        # Open the login URL
        driver.get(login_url)
        time.sleep(2)
        
        # Locate the username and password fields and enter the credentials
        username_field = driver.find_element(By.NAME, "email")
        password_field = driver.find_element(By.NAME, "password")
        username_field.send_keys(username)
        password_field.send_keys(password)
        password_field.send_keys(Keys.RETURN)
        time.sleep(2)
        
        # Click on the button to navigate to the correct page
        button1 = driver.find_element(By.XPATH, "//*[@class='table table-striped']//tr[8]//a")
        button1.click()
        time.sleep(2)
        
        # Click on the download button with the new XPath
        download_button = driver.find_element(By.XPATH, '//*[@id="backtest-container"]/div[2]/a')
        download_button.click()
        time.sleep(5)  # Wait for the download to complete; adjust if needed

        # Define the path to the CSV file in the Downloads folder
        csv_file_path = os.path.join(folder_path, csv_file_name)
        
        # Check if the CSV file exists
        if not os.path.exists(csv_file_path):
            print(f"CSV file not found: {csv_file_path}")
            return ([], None)
        
        # Calculate the current time and round it down to the nearest 15-minute mark
        current_time = dt_datetime.now()
        rounded_time = current_time - timedelta(minutes=current_time.minute % 15,
                                                seconds=current_time.second,
                                                microseconds=current_time.microsecond)
        
        # Function to parse datetime with error handling
        def parse_datetime(date_str):
            try:
                return dt_datetime.strptime(date_str.strip(), '%d-%m-%Y %H:%M')
            except ValueError:
                return None
        
        # Read the CSV file and retrieve stock list
        stockList = []
        with open(csv_file_path, mode='r', newline='', encoding='utf-8-sig') as file:
            reader = csv.reader(file)
            header = next(reader)  # Skip the header if there is one
            for row in reader:
                if len(row) > 0:
                    cell_value = row[0].strip()
                    cell_datetime = parse_datetime(cell_value)
                    if cell_datetime and cell_datetime <= rounded_time:
                        if len(row) > 1:
                            stockList.append(f"{row[1]}-EQ")
        
        return (stockList, rounded_time)
    
    except Exception as e:
        print(f"An error occurred: {e}")
        return ([], None)
    
    finally:
        driver.quit()

# Example usage
chrome_driver_path = r'C:\Users\omkar\AppData\Local\Google\Chrome\User Data\Default\chromedriver-win64\chromedriver.exe'
login_url = "https://chartink.com/login"
username = "akashkharade.760@gmail.com"
password = "7030232281"
csv_file_name = "Backtest BB Blast_Omk, Technical Analysis Scanner.csv"

# Run the script within the time window of 10:14 AM to 11:44 AM
start_time = dt_datetime.now().replace(hour=9, minute=14, second=0, microsecond=0)
end_time = dt_datetime.now().replace(hour=11, minute=44, second=0, microsecond=0)

while True:
    current_time = dt_datetime.now()

    if start_time <= current_time <= end_time:
        # Delete old CSV files
        delete_csv_files()
        
        # Download the new CSV and process it
        stock_list, scan_time = download_csv(chrome_driver_path, login_url, username, password, csv_file_name)
        if scan_time:
            print(f"Stocks scanned for {scan_time.strftime('%I:%M %p')}: {stock_list}")
        else:
            print("No stocks found.")
        
        # Wait for 15 minutes before the next operation
        time.sleep(120)  # 900 seconds = 15 minutes
    elif current_time > end_time:
        print("Time window has passed. Exiting.")
        break
    else:
        # If the current time is before the start time, wait for a minute and check again
        time.sleep(60)


In [ ]:
#### TO KEEP PARTITIAN   ####

In [ ]:
# This script is to get the filter data from Chartink CSV file

import pandas as pd

# Load the CSV file
file_path = r'C:\Users\omkar\Downloads\Backtest BB Blast_Omk, Technical Analysis Scanner.csv'
df = pd.read_csv(file_path)

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')

# Define the time range
start_time = '10:00'
end_time = '11:45'

# Filter data between 10:00 AM and 11:45 AM
filtered_df = df[(df['date'].dt.time >= pd.to_datetime(start_time).time()) &
                 (df['date'].dt.time <= pd.to_datetime(end_time).time())]

# Group by the 'date' and filter out the rows where there are more than 3 stocks for the same timestamp
filtered_df = filtered_df.groupby('date').filter(lambda x: len(x) <= 3)

# Save or print the filtered data
filtered_df.to_csv(r'C:\Users\omkar\Downloads\filtered_stocks.csv', index=False)  # Saves the result to a new CSV
print(filtered_df)


In [ ]:

# this script to get the data of hostory



import datetime
from dateutil.relativedelta import relativedelta
import logging
import pandas as pd
from NorenRestApiPy.NorenApi import NorenApi
import pyotp

class ShoonyaApiPy(NorenApi):
    def __init__(self):
        super().__init__(host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/')

# Configuration Constants
USER = 'FA74468'
PWD = 'GURU222kore!'
TOKEN = 'XT2L66VT73Q22P33BNCHKN6WA2Q37KK6'
VC = 'FA74468_U'
APP_KEY = 'c98e82a190da8181c80fb94cf0a31144'
IMEI = 'abc1234'

# Logging Configuration
logging.basicConfig(
    filename='D:\\AA_trading_Algos\\modified_ALgo\\trading_log.txt',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filemode='w'
)

# Initialize API
api = ShoonyaApiPy()
factor2 = pyotp.TOTP(TOKEN).now()
api.login(userid=USER, password=PWD, twoFA=factor2, vendor_code=VC, api_secret=APP_KEY, imei=IMEI)

# Current date and time
now = datetime.datetime.now()

# Calculate the date 4 months ago
start_date = now - relativedelta(months=4)
start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

# Convert start_date to a timestamp
start_timestamp = start_date.timestamp()

# List of stocks to fetch data for
stocks =['360ONE-EQ', '3MINDIA-EQ', 'ABB-EQ', 'ACC-EQ', 'AIAENG-EQ', 'APLAPOLLO-EQ', 'AUBANK-EQ', 'AARTIIND-EQ', 'AAVAS-EQ', 'ABBOTINDIA-EQ', 'ACE-EQ', 'ADANIENSOL-EQ', 'ADANIENT-EQ', 
         'ADANIGREEN-EQ', 'ADANIPORTS-EQ', 'ADANIPOWER-EQ', 'ATGL-EQ', 'AWL-EQ', 'ABCAPITAL-EQ', 'ABFRL-EQ', 'AEGISCHEM-EQ', 'AETHER-EQ', 'AFFLE-EQ', 'AJANTPHARM-EQ', 'APLLTD-EQ', 'ALKEM-EQ', 
         'ALKYLAMINE-EQ', 'ALLCARGO-EQ', 'ALOKINDS-EQ', 'ARE&M-EQ', 'AMBER-EQ', 'AMBUJACEM-EQ', 'ANANDRATHI-EQ', 'ANGELONE-EQ', 'ANURAS-EQ', 'APARINDS-EQ', 'APOLLOHOSP-EQ', 'APOLLOTYRE-EQ', 'APTUS-EQ', 'ACI-EQ', 'ASAHIINDIA-EQ', 'ASHOKLEY-EQ', 
         'ASIANPAINT-EQ', 'ASTERDM-EQ', 'ASTRAZEN-EQ', 'ASTRAL-EQ', 'ATUL-EQ', 'AUROPHARMA-EQ', 'AVANTIFEED-EQ', 'DMART-EQ', 'AXISBANK-EQ', 'BEML-EQ', 'BLS-EQ', 'BSE-EQ', 'BAJAJ-AUTO-EQ', 'BAJFINANCE-EQ', 'BAJAJFINSV-EQ', 'BAJAJHLDNG-EQ', 'BALAMINES-EQ', 'BALKRISIND-EQ', 
         'BALRAMCHIN-EQ', 'BANDHANBNK-EQ', 'BANKBARODA-EQ', 'BANKINDIA-EQ', 'MAHABANK-EQ', 'BATAINDIA-EQ', 'BAYERCROP-EQ', 'BERGEPAINT-EQ', 'BDL-EQ', 'BEL-EQ', 'BHARATFORG-EQ', 'BHEL-EQ', 'BPCL-EQ', 'BHARTIARTL-EQ', 'BIKAJI-EQ', 'BIOCON-EQ', 'BIRLACORPN-EQ', 'BSOFT-EQ', 'BLUEDART-EQ', 'BLUESTARCO-EQ', 
         'BBTC-EQ', 'BORORENEW-EQ', 'BOSCHLTD-EQ', 'BRIGADE-EQ', 'BRITANNIA-EQ', 'MAPMYINDIA-EQ', 'CCL-EQ', 'CESC-EQ', 'CGPOWER-EQ', 'CIEINDIA-EQ', 'CRISIL-EQ', 'CSBBANK-EQ', 'CAMPUS-EQ', 'CANFINHOME-EQ', 'CANBK-EQ', 'CAPLIPOINT-EQ', 'CGCL-EQ', 'CARBORUNIV-EQ', 'CASTROLIND-EQ', 'CEATLTD-EQ', 'CELLO-EQ',
         'CENTRALBK-EQ', 'CDSL-EQ', 'CENTURYPLY-EQ', 'CENTURYTEX-EQ', 'CERA-EQ', 'CHALET-EQ', 'CHAMBLFERT-EQ', 'CHEMPLASTS-EQ', 'CHENNPETRO-EQ', 'CHOLAHLDNG-EQ', 'CHOLAFIN-EQ', 'CIPLA-EQ', 'CUB-EQ', 'CLEAN-EQ', 'COALINDIA-EQ', 'COCHINSHIP-EQ', 'COFORGE-EQ', 'COLPAL-EQ', 'CAMS-EQ', 'CONCORDBIO-EQ', 'CONCOR-EQ', 
         'COROMANDEL-EQ', 'CRAFTSMAN-EQ', 'CREDITACC-EQ', 'CROMPTON-EQ', 'CUMMINSIND-EQ', 'CYIENT-EQ', 'DCMSHRIRAM-EQ', 'DLF-EQ', 'DOMS-EQ', 'DABUR-EQ', 'DALBHARAT-EQ', 'DATAPATTNS-EQ', 'DEEPAKFERT-EQ', 'DEEPAKNTR-EQ', 'DELHIVERY-EQ', 'DEVYANI-EQ', 'DIVISLAB-EQ', 'DIXON-EQ', 'LALPATHLAB-EQ', 'DRREDDY-EQ', 'EIDPARRY-EQ',
         'EIHOTEL-EQ', 'EPL-EQ', 'EASEMYTRIP-EQ', 'EICHERMOT-EQ', 'ELECON-EQ', 'ELGIEQUIP-EQ', 'EMAMILTD-EQ', 'ENDURANCE-EQ', 'ENGINERSIN-EQ', 'EQUITASBNK-EQ', 'ERIS-EQ', 'ESCORTS-EQ', 'EXIDEIND-EQ', 'FDC-EQ', 'NYKAA-EQ', 'FEDERALBNK-EQ', 'FACT-EQ', 'FINEORG-EQ', 'FINCABLES-EQ', 'FINPIPE-EQ', 'FSL-EQ', 'FIVESTAR-EQ',
         'FORTIS-EQ', 'GAIL-EQ', 'GMMPFAUDLR-EQ', 'GMRINFRA-EQ', 'GRSE-EQ', 'GICRE-EQ', 'GILLETTE-EQ', 'GLAND-EQ', 'GLAXO-EQ', 'GLS-EQ', 'GLENMARK-EQ', 'MEDANTA-EQ', 'GPIL-EQ', 'GODFRYPHLP-EQ', 'GODREJCP-EQ', 'GODREJIND-EQ', 'GODREJPROP-EQ', 'GRANULES-EQ', 'GRAPHITE-EQ', 'GRASIM-EQ', 'GESHIP-EQ', 'GRINDWELL-EQ', 'GAEL-EQ',
         'FLUOROCHEM-EQ', 'GUJGASLTD-EQ', 'GMDCLTD-EQ', 'GNFC-EQ', 'GPPL-EQ', 'GSFC-EQ', 'GSPL-EQ', 'HEG-EQ', 'HBLPOWER-EQ', 'HCLTECH-EQ', 'HDFCAMC-EQ', 'HDFCBANK-EQ', 'HDFCLIFE-EQ', 'HFCL-EQ', 'HAPPSTMNDS-EQ', 'HAPPYFORGE-EQ', 'HAVELLS-EQ', 'HEROMOTOCO-EQ', 'HSCL-EQ', 'HINDALCO-EQ', 'HAL-EQ', 'HINDCOPPER-EQ', 'HINDPETRO-EQ',
         'HINDUNILVR-EQ', 'HINDZINC-EQ', 'POWERINDIA-EQ', 'HOMEFIRST-EQ', 'HONASA-EQ', 'HONAUT-EQ', 'HUDCO-EQ', 'ICICIBANK-EQ', 'ICICIGI-EQ', 'ICICIPRULI-EQ', 'ISEC-EQ', 'IDBI-EQ', 'IDFCFIRSTB-EQ', 'IDFC-EQ', 'IIFL-EQ', 'IRB-EQ', 'IRCON-EQ', 'ITC-EQ', 'ITI-EQ', 'INDIACEM-EQ', 'IBULHSGFIN-EQ', 'INDIAMART-EQ', 'INDIANB-EQ', 'IEX-EQ',
         'INDHOTEL-EQ', 'IOC-EQ', 'IOB-EQ', 'IRCTC-EQ', 'IRFC-EQ', 'INDIGOPNTS-EQ', 'IGL-EQ', 'INDUSTOWER-EQ', 'INDUSINDBK-EQ', 'NAUKRI-EQ', 'INFY-EQ', 'INOXWIND-EQ', 'INTELLECT-EQ', 'INDIGO-EQ', 'IPCALAB-EQ', 'JBCHEPHARM-EQ', 'JKCEMENT-EQ', 'JBMA-EQ', 'JKLAKSHMI-EQ', 'JKPAPER-EQ', 'JMFINANCIL-EQ', 'JSWENERGY-EQ', 'JSWINFRA-EQ', 'JSWSTEEL-EQ', 'JAIBALAJI-EQ', 'J&KBANK-EQ', 'JINDALSAW-EQ', 'JSL-EQ', 'JINDALSTEL-EQ', 'JIOFIN-EQ', 'JUBLFOOD-EQ', 'JUBLINGREA-EQ', 'JUBLPHARMA-EQ', 'JWL-EQ', 'JUSTDIAL-EQ',
         'JYOTHYLAB-EQ', 'KPRMILL-EQ', 'KEI-EQ', 'KNRCON-EQ', 'KPITTECH-EQ', 'KRBL-EQ', 'KSB-EQ', 'KAJARIACER-EQ', 'KPIL-EQ', 'KALYANKJIL-EQ', 'KANSAINER-EQ', 'KARURVYSYA-EQ', 'KAYNES-EQ', 'KEC-EQ', 'KFINTECH-EQ', 'KOTAKBANK-EQ', 'KIMS-EQ', 'L&TFH-EQ', 'LTTS-EQ', 'LICHSGFIN-EQ', 'LTIM-EQ', 'LT-EQ', 'LATENTVIEW-EQ', 'LAURUSLABS-EQ', 'LXCHEM-EQ', 'LEMONTREE-EQ', 'LICI-EQ', 'LINDEINDIA-EQ', 'LLOYDSME-EQ', 'LUPIN-EQ', 'MMTC-EQ', 'MRF-EQ', 'MTARTECH-EQ', 'LODHA-EQ', 'MGL-EQ', 'MAHSEAMLES-EQ', 'M&MFIN-EQ', 'M&M-EQ', 'MHRIL-EQ', 'MAHLIFE-EQ', 'MANAPPURAM-EQ', 'MRPL-EQ', 'MANKIND-EQ', 'MARICO-EQ', 'MARUTI-EQ', 'MASTEK-EQ', 'MFSL-EQ', 'MAXHEALTH-EQ', 'MAZDOCK-EQ', 'MEDPLUS-EQ', 'METROBRAND-EQ', 'METROPOLIS-EQ', 'MINDACORP-EQ', 'MSUMI-EQ', 'MOTILALOFS-EQ', 'MPHASIS-EQ', 'MCX-EQ', 'MUTHOOTFIN-EQ', 'NATCOPHARM-EQ', 'NBCC-EQ', 'NCC-EQ', 'NHPC-EQ', 'NLCINDIA-EQ', 'NMDC-EQ', 'NSLNISP-EQ', 'NTPC-EQ', 'NH-EQ', 'NATIONALUM-EQ', 'NAVINFLUOR-EQ', 'NESTLEIND-EQ', 'NETWORK18-EQ', 'NAM-INDIA-EQ', 'NUVAMA-EQ', 'NUVOCO-EQ', 'OBEROIRLTY-EQ', 'ONGC-EQ', 'OIL-EQ', 'OLECTRA-EQ', 'PAYTM-EQ', 'OFSS-EQ', 'POLICYBZR-EQ', 'PCBL-EQ', 'PIIND-EQ', 'PNBHOUSING-EQ', 'PNCINFRA-EQ', 'PVRINOX-EQ', 'PAGEIND-EQ',
         'PATANJALI-EQ', 'PERSISTENT-EQ', 'PETRONET-EQ', 'PHOENIXLTD-EQ', 'PIDILITIND-EQ', 'PEL-EQ', 'PPLPHARMA-EQ', 'POLYMED-EQ', 'POLYCAB-EQ', 'POONAWALLA-EQ', 'PFC-EQ', 'POWERGRID-EQ', 'PRAJIND-EQ', 'PRESTIGE-EQ', 'PRINCEPIPE-EQ', 'PRSMJOHNSN-EQ', 'PGHH-EQ', 'PNB-EQ', 'QUESS-EQ', 'RRKABEL-EQ', 'RBLBANK-EQ', 'RECLTD-EQ', 'RHIM-EQ', 'RITES-EQ', 'RADICO-EQ', 'RVNL-EQ', 'RAILTEL-EQ', 'RAINBOW-EQ', 'RAJESHEXPO-EQ', 'RKFORGE-EQ', 'RCF-EQ', 'RATNAMANI-EQ', 'RTNINDIA-EQ', 'RAYMOND-EQ', 'REDINGTON-EQ', 'RELIANCE-EQ', 'RBA-EQ', 'ROUTE-EQ', 'SBFC-EQ', 'SBICARD-EQ', 'SBILIFE-EQ', 'SJVN-EQ', 'SKFINDIA-EQ', 'SRF-EQ', 'SAFARI-EQ', 'MOTHERSON-EQ', 'SANOFI-EQ', 'SAPPHIRE-EQ', 'SAREGAMA-EQ', 'SCHAEFFLER-EQ', 'SCHNEIDER-EQ', 'SHREECEM-EQ', 'RENUKA-EQ', 'SHRIRAMFIN-EQ', 'SHYAMMETL-EQ', 'SIEMENS-EQ', 'SIGNATURE-EQ', 'SOBHA-EQ', 'SOLARINDS-EQ', 'SONACOMS-EQ', 'SONATSOFTW-EQ', 'STARHEALTH-EQ', 'SBIN-EQ', 'SAIL-EQ', 'SWSOLAR-EQ', 'STLTECH-EQ', 'SUMICHEM-EQ', 'SPARC-EQ', 'SUNPHARMA-EQ', 'SUNTV-EQ', 'SUNDARMFIN-EQ', 'SUNDRMFAST-EQ', 'SUNTECK-EQ', 'SUPREMEIND-EQ', 'SUVENPHAR-EQ', 'SUZLON-EQ',
         'SWANENERGY-EQ', 'SYNGENE-EQ', 'SYRMA-EQ', 'TV18BRDCST-EQ', 'TVSMOTOR-EQ', 'TVSSCS-EQ', 'TMB-EQ', 'TANLA-EQ', 'TATACHEM-EQ', 'TATACOMM-EQ', 'TCS-EQ', 'TATACONSUM-EQ', 'TATAELXSI-EQ', 'TATAINVEST-EQ', 'TATAMTRDVR-EQ', 'TATAMOTORS-EQ', 'TATAPOWER-EQ', 'TATASTEEL-EQ', 'TATATECH-EQ', 'TTML-EQ', 'TECHM-EQ', 'TEJASNET-EQ', 'NIACL-EQ', 'RAMCOCEM-EQ', 'THERMAX-EQ', 'TIMKEN-EQ', 'TITAGARH-EQ', 'TITAN-EQ', 'TORNTPHARM-EQ', 'TORNTPOWER-EQ', 'TRENT-EQ', 'TRIDENT-EQ', 'TRIVENI-EQ', 'TRITURBINE-EQ', 'TIINDIA-EQ', 'UCOBANK-EQ', 'UNOMINDA-EQ', 'UPL-EQ', 'UTIAMC-EQ', 'UJJIVANSFB-EQ', 'ULTRACEMCO-EQ', 'UNIONBANK-EQ', 'UBL-EQ', 'MCDOWELL-N-EQ', 'USHAMART-EQ', 'VGUARD-EQ', 'VIPIND-EQ', 'VAIBHAVGBL-EQ', 'VTL-EQ', 'VARROC-EQ', 'VBL-EQ', 'MANYAVAR-EQ', 'VEDL-EQ', 
         'VIJAYA-EQ', 'IDEA-EQ', 'VOLTAS-EQ', 'WELCORP-EQ', 'WELSPUNLIV-EQ', 'WESTLIFE-EQ', 'WHIRLPOOL-EQ', 'WIPRO-EQ', 'YESBANK-EQ', 'ZFCVINDIA-EQ', 'ZEEL-EQ', 'ZENSARTECH-EQ', 'ZOMATO-EQ', 'ZYDUSLIFE-EQ', 'ECLERX-EQ']

for stock in stocks:
    try:
        # Fetch the data from the API for each stock
        ret = api.get_time_price_series(exchange='NSE', token=stock, starttime=start_timestamp, interval=3)

        # Convert the response into a DataFrame
        df = pd.DataFrame(ret)

        # Save the DataFrame to an Excel file named after the stock
        output_file_path = f'C:\\Users\\omkar\\Downloads\\{stock}.xlsx'
        df.to_excel(output_file_path, index=False)

        print(f"Data for {stock} saved to {output_file_path}")

    except Exception as e:
        logging.error(f"An error occurred while fetching data for {stock}: {e}")


In [ ]:
import pandas as pd
import os

# Set the directory where the Excel sheets are stored
excel_directory = 'C:\\Users\\omkar\\Downloads\\Dwnladsheet'

# stocks_info = [('23-09-2024 10:30', 'JIOFIN'),
stocks_info = [
    ('07-06-2024 10:00', 'COCHINSHIP'),
    ('07-06-2024 10:00', 'SUNPHARMA'),
    ('07-06-2024 10:00', 'ABCAPITAL'),
    ('10-06-2024 10:00', 'NATCOPHARM'),
    ('10-06-2024 10:00', 'MAXHEALTH'),
    ('10-06-2024 11:00', 'HAVELLS'),
    ('12-06-2024 10:00', 'HAPPSTMNDS'),
    ('12-06-2024 10:00', 'PETRONET'),
    ('12-06-2024 10:00', 'GPPL'),
    ('13-06-2024 10:00', 'AUBANK'),
    ('13-06-2024 10:00', 'FSL'),
    ('14-06-2024 10:45', 'RBLBANK'),
    ('18-06-2024 10:15', 'ADANIENT'),
    ('18-06-2024 10:15', 'NTPC'),
    ('18-06-2024 10:15', 'RAILTEL'),
    ('18-06-2024 10:30', 'M&M'),
    ('19-06-2024 10:00', 'GSFC'),
    ('19-06-2024 10:00', 'KALYANKJIL'),
    ('19-06-2024 10:15', 'COROMANDEL'),
    ('19-06-2024 10:30', 'ICICIBANK'),
    ('19-06-2024 10:30', 'INDUSINDBK'),
    ('20-06-2024 10:00', 'KOTAKBANK'),
    ('21-06-2024 10:15', 'INFY'),
    ('21-06-2024 10:15', 'HBLPOWER'),
    ('21-06-2024 10:30', 'RECLTD'),
    ('24-06-2024 10:15', 'M&MFIN'),
    ('25-06-2024 10:15', 'ZOMATO'),
    ('25-06-2024 10:45', 'MSUMI'),
    ('25-06-2024 11:30', 'PPLPHARMA'),
    ('26-06-2024 10:00', 'PEL'),
    ('26-06-2024 10:00', 'MANAPPURAM'),
    ('27-06-2024 10:00', 'PFC'),
    ('27-06-2024 10:30', 'AEGISLOG'),
    ('27-06-2024 10:30', 'ITI'),
    ('27-06-2024 10:45', 'WESTLIFE'),
    ('27-06-2024 10:45', 'GICRE'),
    ('28-06-2024 10:00', 'ONGC'),
    ('28-06-2024 10:00', 'PETRONET'),
    ('28-06-2024 10:15', 'ZOMATO'),
    ('28-06-2024 10:15', 'POLICYBZR'),
    ('28-06-2024 10:30', 'AETHER'),
    ('01-07-2024 10:00', 'BEL'),
    ('01-07-2024 10:00', 'COCHINSHIP'),
    ('01-07-2024 10:15', 'EPL'),
    ('01-07-2024 10:30', 'NETWORK18'),
    ('02-07-2024 10:00', 'RITES'),
    ('02-07-2024 10:00', 'CUB'),
    ('02-07-2024 10:15', 'ONGC'),
    ('02-07-2024 10:30', 'MRPL'),
    ('02-07-2024 10:30', 'KSB'),
    ('02-07-2024 11:30', 'NATIONALUM'),
    ('03-07-2024 10:30', 'HSCL'),
    ('03-07-2024 10:45', 'RCF'),
    ('03-07-2024 10:45', 'TVSSCS'),
    ('04-07-2024 10:15', 'HAVELLS'),
    ('04–07–2024      ','NHPC'), 
   ...
   # Continue adding all events in the same format
]


# Initialize a list to hold results
results = []

# Iterate over each stock entry
for entry_time_str, stock_name in stocks_info:
    excel_file_name = f"{stock_name}-EQ.xlsx"
    excel_file_path = os.path.join(excel_directory, excel_file_name)

    try:
        # Load stock data
        stock_data = pd.read_excel(excel_file_path, usecols=['time', 'intc'])
        stock_data['time'] = pd.to_datetime(stock_data['time'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
        stock_data.sort_values(by='time', ascending=True, inplace=True)

        # Convert entry_time_str to timestamp
        entry_time = pd.to_datetime(entry_time_str, format='%d-%m-%Y %H:%M')

        # Entry row
        entry_row = stock_data[stock_data['time'] == entry_time]

        if not entry_row.empty:
            entry_price = entry_row['intc'].values[0]
            profit_target = entry_price * 0.992  # 0.8% profit target
            stop_loss = entry_price * 1.0045      # 0.45% stop loss

            # Start checking from the entry time
            subsequent_data = stock_data[stock_data['time'] > entry_time]

            for index, row in subsequent_data.iterrows():
                current_price = row['intc']
                current_time = row['time']

                if current_price < profit_target:
                    results.append([stock_name, current_time, current_price, 'Profit'])
                    print(f"Profit target hit at {current_time}: {current_price:.2f}")
                    break
                elif current_price > stop_loss:
                    results.append([stock_name, current_time, current_price, 'Loss'])
                    print(f"Stop loss hit at {current_time}: {current_price:.2f}")
                    break
            else:
                print("Neither SL nor TGT hit within the checked time frame.")
        else:
            print(f"No entry found for {stock_name} at {entry_time_str}")

    except Exception as e:
        print(f"An error occurred for {stock_name}: {e}")

# Create a DataFrame for the results
results_df = pd.DataFrame(results, columns=['Stock Name', 'Entry Time', 'Price', 'Status'])

# Save the results to an Excel file
output_file_path = 'C:\\Users\\omkar\\Downloads\\stock_results.xlsx'
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")
print(f"Total results: {len(results)}")


In [ ]:
def format_data(input_data):
    """
    Convert a multiline string of stock data into an array of tuples.
    
    Args:
        input_data (str): A multiline string of stock data.
    
    Returns:
        list: A list of tuples containing the formatted data.
    """
    formatted_data = []
    for line in input_data.strip().split('\n'):
        # Split by the last tab character to separate date_time and company
        parts = line.rsplit('\t', 1)
        if len(parts) == 2:
            date_time = parts[0].strip()  # Remove any extra whitespace
            company = parts[1].strip()     # Remove any extra whitespace
            formatted_data.append((date_time, company))
    return formatted_data

# Example input data (you can change this as needed)
data = """
07-06-2024 10:00	COCHINSHIP
07-06-2024 10:00	SUNPHARMA
07-06-2024 10:00	ABCAPITAL
10-06-2024 10:00	NATCOPHARM
10-06-2024 10:00	MAXHEALTH
10-06-2024 11:00	HAVELLS
12-06-2024 10:00	HAPPSTMNDS
12-06-2024 10:00	PETRONET
12-06-2024 10:00	GPPL
13-06-2024 10:00	AUBANK
13-06-2024 10:00	FSL
14-06-2024 10:45	RBLBANK
18-06-2024 10:15	ADANIENT
18-06-2024 10:15	NTPC
18-06-2024 10:15	RAILTEL
18-06-2024 10:30	M&M
19-06-2024 10:00	GSFC
19-06-2024 10:00	KALYANKJIL
19-06-2024 10:15	COROMANDEL
19-06-2024 10:30	ICICIBANK
19-06-2024 10:30	INDUSINDBK
20-06-2024 10:00	KOTAKBANK
21-06-2024 10:15	INFY
21-06-2024 10:15	HBLPOWER
21-06-2024 10:30	RECLTD
24-06-2024 10:15	M&MFIN
25-06-2024 10:15	ZOMATO
25-06-2024 10:45	MSUMI
25-06-2024 11:30	PPLPHARMA
26-06-2024 10:00	PEL
26-06-2024 10:00	MANAPPURAM
27-06-2024 10:00	PFC
27-06-2024 10:30	AEGISLOG
27-06-2024 10:30	ITI
27-06-2024 10:45	WESTLIFE
27-06-2024 10:45	GICRE
28-06-2024 10:00	ONGC
28-06-2024 10:00	PETRONET
28-06-2024 10:15	ZOMATO
28-06-2024 10:15	POLICYBZR
28-06-2024 10:30	AETHER
01-07-2024 10:00	BEL
01-07-2024 10:00	COCHINSHIP
01-07-2024 10:15	EPL
01-07-2024 10:30	NETWORK18
02-07-2024 10:00	RITES
02-07-2024 10:00	CUB
02-07-2024 10:15	ONGC
02-07-2024 10:30	MRPL
02-07-2024 10:30	KSB
02-07-2024 11:30	NATIONALUM
03-07-2024 10:30	HSCL
03-07-2024 10:45	RCF
03-07-2024 10:45	TVSSCS
04-07-2024 10:15	HAVELLS
04-07-2024 10:15	NHPC
04-07-2024 10:30	KOTAKBANK
04-07-2024 10:30	CONCOR
04-07-2024 10:30	SJVN
04-07-2024 11:00	ABFRL
05-07-2024 10:00	PERSISTENT
05-07-2024 11:15	EPL
08-07-2024 10:00	TATAMTRDVR
08-07-2024 10:00	TATAMOTORS
08-07-2024 10:00	UNOMINDA
08-07-2024 10:15	PCBL
08-07-2024 10:15	VGUARD
08-07-2024 10:30	AFFLE
09-07-2024 10:00	MSUMI
09-07-2024 10:00	ADANIENT
09-07-2024 10:00	CDSL
11-07-2024 10:00	ACC
11-07-2024 10:00	BPCL
11-07-2024 10:15	TATAPOWER
11-07-2024 10:15	MANKIND
11-07-2024 10:15	M&MFIN
15-07-2024 10:00	PPLPHARMA
15-07-2024 10:15	HCLTECH
16-07-2024 10:00	BALRAMCHIN
16-07-2024 10:00	MRPL
16-07-2024 10:00	KALYANKJIL
16-07-2024 11:30	LICHSGFIN
18-07-2024 10:00	TORNTPOWER
18-07-2024 10:15	HINDPETRO
18-07-2024 10:15	RAJESHEXPO
18-07-2024 10:45	SBIN
19-07-2024 11:00	AVANTIFEED
19-07-2024 11:00	JUBLINGREA
23-07-2024 10:00	JSWENERGY
23-07-2024 10:45	GLAXO
24-07-2024 10:00	ITC
26-07-2024 10:00	NCC
26-07-2024 10:00	JSWINFRA
26-07-2024 10:00	NHPC
26-07-2024 10:15	STLTECH
26-07-2024 10:15	INDIANB
26-07-2024 10:30	SAREGAMA
26-07-2024 11:15	AFFLE
29-07-2024 10:15	ISEC
29-07-2024 10:30	GSFC
29-07-2024 10:30	NETWORK18
29-07-2024 10:45	SAMMAANCAP
30-07-2024 10:00	SAREGAMA
30-07-2024 10:00	NHPC
31-07-2024 10:00	EPL
31-07-2024 10:00	TATAPOWER
31-07-2024 10:00	GMRINFRA
31-07-2024 10:15	HSCL
31-07-2024 10:15	TRITURBINE
31-07-2024 10:30	CDSL
01-08-2024 10:00	VEDL
01-08-2024 10:00	AUBANK
01-08-2024 10:15	COALINDIA
01-08-2024 10:30	GMDCLTD
01-08-2024 10:30	M&MFIN
01-08-2024 10:30	JSWENERGY
02-08-2024 10:00	GODREJCP
08-08-2024 10:00	BIKAJI
12-08-2024 10:00	CDSL
12-08-2024 10:00	RAILTEL
13-08-2024 10:00	ADANIENSOL
13-08-2024 10:15	JIOFIN
13-08-2024 10:15	RVNL
16-08-2024 10:15	TATAMTRDVR
16-08-2024 10:15	BPCL
16-08-2024 10:15	RAILTEL
19-08-2024 10:45	SAREGAMA
20-08-2024 10:30	NIACL
20-08-2024 11:15	ZENSARTECH
20-08-2024 11:15	GAEL
21-08-2024 10:00	COCHINSHIP
21-08-2024 11:00	DEVYANI
22-08-2024 10:15	BANDHANBNK
23-08-2024 10:00	SHYAMMETL
23-08-2024 10:15	TVSMOTOR
23-08-2024 10:30	RITES
26-08-2024 10:00	BIKAJI
26-08-2024 10:30	MAHSEAMLES
26-08-2024 10:30	HBLPOWER
26-08-2024 10:30	INOXWIND
26-08-2024 11:30	INDHOTEL
27-08-2024 10:00	GESHIP
27-08-2024 10:00	OIL
27-08-2024 10:00	SONACOMS
27-08-2024 10:15	RCF
27-08-2024 10:30	HAVELLS
28-08-2024 10:15	PAYTM
28-08-2024 11:15	INTELLECT
30-08-2024 10:00	EIDPARRY
30-08-2024 10:15	BALRAMCHIN
30-08-2024 10:15	COLPAL
30-08-2024 10:15	GAIL
02-09-2024 10:15	JMFINANCIL
02-09-2024 10:15	UNOMINDA
02-09-2024 10:30	HSCL
02-09-2024 10:30	SUVENPHAR
02-09-2024 10:45	PCBL
03-09-2024 10:00	HONASA
03-09-2024 10:00	INOXWIND
03-09-2024 10:15	LUPIN
03-09-2024 10:30	BSOFT
03-09-2024 11:00	M&MFIN
04-09-2024 10:00	SBFC
04-09-2024 10:15	KPIL
04-09-2024 10:30	WESTLIFE
05-09-2024 10:15	HINDZINC
05-09-2024 10:15	JSWENERGY
05-09-2024 10:30	CEATLTD
05-09-2024 10:45	RITES
05-09-2024 11:30	SUNTECK
06-09-2024 10:15	AAVAS
06-09-2024 10:15	JYOTHYLAB
10-09-2024 10:15	BHARTIARTL
10-09-2024 10:15	GMRINFRA
11-09-2024 10:15	JYOTHYLAB
11-09-2024 10:15	LXCHEM
11-09-2024 10:45	BERGEPAINT
12-09-2024 10:00	BEL
12-09-2024 10:00	INDIANB
12-09-2024 10:15	VBL
12-09-2024 10:45	BPCL
13-09-2024 10:00	IRCTC
13-09-2024 10:00	SHYAMMETL
13-09-2024 10:15	HAVELLS
13-09-2024 10:15	CDSL
13-09-2024 10:15	RTNINDIA
13-09-2024 11:00	POONAWALLA
13-09-2024 11:30	STLTECH
16-09-2024 10:15	DELHIVERY
16-09-2024 10:15	J&KBANK
16-09-2024 10:45	INDIACEM
17-09-2024 10:00	FSL
17-09-2024 10:15	TORNTPOWER
18-09-2024 10:00	SYRMA
18-09-2024 10:00	NYKAA
18-09-2024 10:15	POONAWALLA
19-09-2024 10:30	SUNTECK
20-09-2024 10:00	NMDC
20-09-2024 10:00	UNIONBANK
20-09-2024 10:15	FORTIS
20-09-2024 10:30	TATACONSUM
20-09-2024 10:45	JWL
23-09-2024 10:30	JIOFIN
24-09-2024 10:00	CESC
24-09-2024 10:00	NESTLEIND
24-09-2024 10:00	LICI
25-09-2024 10:00	MAZDOCK
"""

# Call the function with the example data and print results in desired format.
formatted_results = format_data(data)

# Print the results in a structured format.
for result in formatted_results:
    print(f"    {result},")

In [ ]:

# Updated code for the testing 



import pandas as pd
import os

# Set the directory where the Excel sheets are stored
excel_directory = 'C:\\Users\\omkar\\Downloads\\Dwnladsheet'

# stocks_info = [('23-09-2024 10:30', 'JIOFIN'),
stocks_info = [
   
     ('07-06-2024 10:00', 'COCHINSHIP'),
    ('07-06-2024 10:00', 'SUNPHARMA'),
    ('07-06-2024 10:00', 'ABCAPITAL'),
    ('10-06-2024 10:00', 'NATCOPHARM'),
    ('10-06-2024 10:00', 'MAXHEALTH'),
    ('10-06-2024 11:00', 'HAVELLS'),
    ('12-06-2024 10:00', 'HAPPSTMNDS'),
    ('12-06-2024 10:00', 'PETRONET'),
    ('12-06-2024 10:00', 'GPPL'),
    ('13-06-2024 10:00', 'AUBANK'),
    ('13-06-2024 10:00', 'FSL'),
    ('14-06-2024 10:45', 'RBLBANK'),
    ('18-06-2024 10:15', 'ADANIENT'),
    ('18-06-2024 10:15', 'NTPC'),
    ('18-06-2024 10:15', 'RAILTEL'),
    ('18-06-2024 10:30', 'M&M'),
    ('19-06-2024 10:00', 'GSFC'),
    ('19-06-2024 10:00', 'KALYANKJIL'),
    ('19-06-2024 10:15', 'COROMANDEL'),
    ('19-06-2024 10:30', 'ICICIBANK'),
    ('19-06-2024 10:30', 'INDUSINDBK'),
    ('20-06-2024 10:00', 'KOTAKBANK'),
    ('21-06-2024 10:15', 'INFY'),
    ('21-06-2024 10:15', 'HBLPOWER'),
    ('21-06-2024 10:30', 'RECLTD'),
    ('24-06-2024 10:15', 'M&MFIN'),
    ('25-06-2024 10:15', 'ZOMATO'),
    ('25-06-2024 10:45', 'MSUMI'),
    ('25-06-2024 11:30', 'PPLPHARMA'),
    ('26-06-2024 10:00', 'PEL'),
    ('26-06-2024 10:00', 'MANAPPURAM'),
    ('27-06-2024 10:00', 'PFC'),
    ('27-06-2024 10:30', 'AEGISLOG'),
    ('27-06-2024 10:30', 'ITI'),
    ('27-06-2024 10:45', 'WESTLIFE'),
    ('27-06-2024 10:45', 'GICRE'),
    ('28-06-2024 10:00', 'ONGC'),
    ('28-06-2024 10:00', 'PETRONET'),
    ('28-06-2024 10:15', 'ZOMATO'),
    ('28-06-2024 10:15', 'POLICYBZR'),
    ('28-06-2024 10:30', 'AETHER'),
    ('01-07-2024 10:00', 'BEL'),
    ('01-07-2024 10:00', 'COCHINSHIP'),
    ('01-07-2024 10:15', 'EPL'),
    ('01-07-2024 10:30', 'NETWORK18'),
    ('02-07-2024 10:00', 'RITES'),
    ('02-07-2024 10:00', 'CUB'),
    ('02-07-2024 10:15', 'ONGC'),
    ('02-07-2024 10:30', 'MRPL'),
    ('02-07-2024 10:30', 'KSB'),
    ('02-07-2024 11:30', 'NATIONALUM'),
    ('03-07-2024 10:30', 'HSCL'),
    ('03-07-2024 10:45', 'RCF'),
    ('03-07-2024 10:45', 'TVSSCS'),
    ('04-07-2024 10:15', 'HAVELLS'),
    ('04-07-2024 10:15', 'NHPC'),
    ('04-07-2024 10:30', 'KOTAKBANK'),
    ('04-07-2024 10:30', 'CONCOR'),
    ('04-07-2024 10:30', 'SJVN'),
    ('04-07-2024 11:00', 'ABFRL'),
    ('05-07-2024 10:00', 'PERSISTENT'),
    ('05-07-2024 11:15', 'EPL'),
    ('08-07-2024 10:00', 'TATAMTRDVR'),
    ('08-07-2024 10:00', 'TATAMOTORS'),
    ('08-07-2024 10:00', 'UNOMINDA'),
    ('08-07-2024 10:15', 'PCBL'),
    ('08-07-2024 10:15', 'VGUARD'),
    ('08-07-2024 10:30', 'AFFLE'),
    ('09-07-2024 10:00', 'MSUMI'),
    ('09-07-2024 10:00', 'ADANIENT'),
    ('09-07-2024 10:00', 'CDSL'),
    ('11-07-2024 10:00', 'ACC'),
    ('11-07-2024 10:00', 'BPCL'),
    ('11-07-2024 10:15', 'TATAPOWER'),
    ('11-07-2024 10:15', 'MANKIND'),
    ('11-07-2024 10:15', 'M&MFIN'),
    ('15-07-2024 10:00', 'PPLPHARMA'),
    ('15-07-2024 10:15', 'HCLTECH'),
    ('16-07-2024 10:00', 'BALRAMCHIN'),
    ('16-07-2024 10:00', 'MRPL'),
    ('16-07-2024 10:00', 'KALYANKJIL'),
    ('16-07-2024 11:30', 'LICHSGFIN'),
    ('18-07-2024 10:00', 'TORNTPOWER'),
    ('18-07-2024 10:15', 'HINDPETRO'),
    ('18-07-2024 10:15', 'RAJESHEXPO'),
    ('18-07-2024 10:45', 'SBIN'),
    ('19-07-2024 11:00', 'AVANTIFEED'),
    ('19-07-2024 11:00', 'JUBLINGREA'),
    ('23-07-2024 10:00', 'JSWENERGY'),
    ('23-07-2024 10:45', 'GLAXO'),
    ('24-07-2024 10:00', 'ITC'),
    ('26-07-2024 10:00', 'NCC'),
    ('26-07-2024 10:00', 'JSWINFRA'),
    ('26-07-2024 10:00', 'NHPC'),
    ('26-07-2024 10:15', 'STLTECH'),
    ('26-07-2024 10:15', 'INDIANB'),
    ('26-07-2024 10:30', 'SAREGAMA'),
    ('26-07-2024 11:15', 'AFFLE'),
    ('29-07-2024 10:15', 'ISEC'),
    ('29-07-2024 10:30', 'GSFC'),
    ('29-07-2024 10:30', 'NETWORK18'),
    ('29-07-2024 10:45', 'SAMMAANCAP'),
    ('30-07-2024 10:00', 'SAREGAMA'),
    ('30-07-2024 10:00', 'NHPC'),
    ('31-07-2024 10:00', 'EPL'),
    ('31-07-2024 10:00', 'TATAPOWER'),
    ('31-07-2024 10:00', 'GMRINFRA'),
    ('31-07-2024 10:15', 'HSCL'),
    ('31-07-2024 10:15', 'TRITURBINE'),
    ('31-07-2024 10:30', 'CDSL'),
    ('01-08-2024 10:00', 'VEDL'),
    ('01-08-2024 10:00', 'AUBANK'),
    ('01-08-2024 10:15', 'COALINDIA'),
    ('01-08-2024 10:30', 'GMDCLTD'),
    ('01-08-2024 10:30', 'M&MFIN'),
    ('01-08-2024 10:30', 'JSWENERGY'),
    ('02-08-2024 10:00', 'GODREJCP'),
    ('08-08-2024 10:00', 'BIKAJI'),
    ('12-08-2024 10:00', 'CDSL'),
    ('12-08-2024 10:00', 'RAILTEL'),
    ('13-08-2024 10:00', 'ADANIENSOL'),
    ('13-08-2024 10:15', 'JIOFIN'),
    ('13-08-2024 10:15', 'RVNL'),
    ('16-08-2024 10:15', 'TATAMTRDVR'),
    ('16-08-2024 10:15', 'BPCL'),
    ('16-08-2024 10:15', 'RAILTEL'),
    ('19-08-2024 10:45', 'SAREGAMA'),
    ('20-08-2024 10:30', 'NIACL'),
    ('20-08-2024 11:15', 'ZENSARTECH'),
    ('20-08-2024 11:15', 'GAEL'),
    ('21-08-2024 10:00', 'COCHINSHIP'),
    ('21-08-2024 11:00', 'DEVYANI'),
    ('22-08-2024 10:15', 'BANDHANBNK'),
    ('23-08-2024 10:00', 'SHYAMMETL'),
    ('23-08-2024 10:15', 'TVSMOTOR'),
    ('23-08-2024 10:30', 'RITES'),
    ('26-08-2024 10:00', 'BIKAJI'),
    ('26-08-2024 10:30', 'MAHSEAMLES'),
    ('26-08-2024 10:30', 'HBLPOWER'),
    ('26-08-2024 10:30', 'INOXWIND'),
    ('26-08-2024 11:30', 'INDHOTEL'),
    ('27-08-2024 10:00', 'GESHIP'),
    ('27-08-2024 10:00', 'OIL'),
    ('27-08-2024 10:00', 'SONACOMS'),
    ('27-08-2024 10:15', 'RCF'),
    ('27-08-2024 10:30', 'HAVELLS'),
    ('28-08-2024 10:15', 'PAYTM'),
    ('28-08-2024 11:15', 'INTELLECT'),
    ('30-08-2024 10:00', 'EIDPARRY'),
    ('30-08-2024 10:15', 'BALRAMCHIN'),
    ('30-08-2024 10:15', 'COLPAL'),
    ('30-08-2024 10:15', 'GAIL'),
    ('02-09-2024 10:15', 'JMFINANCIL'),
    ('02-09-2024 10:15', 'UNOMINDA'),
    ('02-09-2024 10:30', 'HSCL'),
    ('02-09-2024 10:30', 'SUVENPHAR'),
    ('02-09-2024 10:45', 'PCBL'),
    ('03-09-2024 10:00', 'HONASA'),
    ('03-09-2024 10:00', 'INOXWIND'),
    ('03-09-2024 10:15', 'LUPIN'),
    ('03-09-2024 10:30', 'BSOFT'),
    ('03-09-2024 11:00', 'M&MFIN'),
    ('04-09-2024 10:00', 'SBFC'),
    ('04-09-2024 10:15', 'KPIL'),
    ('04-09-2024 10:30', 'WESTLIFE'),
    ('05-09-2024 10:15', 'HINDZINC'),
    ('05-09-2024 10:15', 'JSWENERGY'),
    ('05-09-2024 10:30', 'CEATLTD'),
    ('05-09-2024 10:45', 'RITES'),
    ('05-09-2024 11:30', 'SUNTECK'),
    ('06-09-2024 10:15', 'AAVAS'),
    ('06-09-2024 10:15', 'JYOTHYLAB'),
    ('10-09-2024 10:15', 'BHARTIARTL'),
    ('10-09-2024 10:15', 'GMRINFRA'),
    ('11-09-2024 10:15', 'JYOTHYLAB'),
    ('11-09-2024 10:15', 'LXCHEM'),
    ('11-09-2024 10:45', 'BERGEPAINT'),
    ('12-09-2024 10:00', 'BEL'),
    ('12-09-2024 10:00', 'INDIANB'),
    ('12-09-2024 10:15', 'VBL'),
    ('12-09-2024 10:45', 'BPCL'),
    ('13-09-2024 10:00', 'IRCTC'),
    ('13-09-2024 10:00', 'SHYAMMETL'),
    ('13-09-2024 10:15', 'HAVELLS'),
    ('13-09-2024 10:15', 'CDSL'),
    ('13-09-2024 10:15', 'RTNINDIA'),
    ('13-09-2024 11:00', 'POONAWALLA'),
    ('13-09-2024 11:30', 'STLTECH'),
    ('16-09-2024 10:15', 'DELHIVERY'),
    ('16-09-2024 10:15', 'J&KBANK'),
    ('16-09-2024 10:45', 'INDIACEM'),
    ('17-09-2024 10:00', 'FSL'),
    ('17-09-2024 10:15', 'TORNTPOWER'),
    ('18-09-2024 10:00', 'SYRMA'),
    ('18-09-2024 10:00', 'NYKAA'),
    ('18-09-2024 10:15', 'POONAWALLA'),
    ('19-09-2024 10:30', 'SUNTECK'),
    ('20-09-2024 10:00', 'NMDC'),
    ('20-09-2024 10:00', 'UNIONBANK'),
    ('20-09-2024 10:15', 'FORTIS'),
    ('20-09-2024 10:30', 'TATACONSUM'),
    ('20-09-2024 10:45', 'JWL'),
    ('23-09-2024 10:30', 'JIOFIN'),
    ('24-09-2024 10:00', 'CESC'),
    ('24-09-2024 10:00', 'NESTLEIND'),
    ('24-09-2024 10:00', 'LICI'),
    ('25-09-2024 10:00', 'MAZDOCK'),

]



results = []

# Assume initial investment for each stock
initial_investment = 50000

# Iterate over each stock entry
for entry_time_str, stock_name in stocks_info:
    excel_file_name = f"{stock_name}-EQ.xlsx"
    excel_file_path = os.path.join(excel_directory, excel_file_name)

    try:
        # Load stock data
        stock_data = pd.read_excel(excel_file_path, usecols=['time', 'intc'])
        stock_data['time'] = pd.to_datetime(stock_data['time'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
        stock_data.sort_values(by='time', ascending=True, inplace=True)

        # Convert entry_time_str to timestamp
        entry_time = pd.to_datetime(entry_time_str, format='%d-%m-%Y %H:%M')

        # Entry row
        entry_row = stock_data[stock_data['time'] == entry_time]

        if not entry_row.empty:
            entry_price = entry_row['intc'].values[0]
            qty = int(initial_investment / entry_price)  # Calculate quantity of stocks

            profit_target = entry_price * 0.992  # 0.8% profit target
            stop_loss = entry_price * 1.0045      # 0.45% stop loss

            # Start checking from the entry time
            subsequent_data = stock_data[stock_data['time'] > entry_time]

            for index, row in subsequent_data.iterrows():
                current_price = row['intc']
                current_time = row['time']

                if current_price < profit_target:
                    profit_loss_amount = (current_price - entry_price) * qty
                    results.append([stock_name, current_time, current_price, 'Profit', profit_loss_amount])
                    print(f"Profit target hit at {current_time}: {current_price:.2f}, Profit: {profit_loss_amount:.2f}")
                    break
                elif current_price > stop_loss:
                    profit_loss_amount = (current_price - entry_price) * qty
                    results.append([stock_name, current_time, current_price, 'Loss', profit_loss_amount])
                    print(f"Stop loss hit at {current_time}: {current_price:.2f}, Loss: {profit_loss_amount:.2f}")
                    break
            else:
                print("Neither SL nor TGT hit within the checked time frame.")
        else:
            print(f"No entry found for {stock_name} at {entry_time_str}")

    except Exception as e:
        print(f"An error occurred for {stock_name}: {e}")

# Create a DataFrame for the results
results_df = pd.DataFrame(results, columns=['Stock Name', 'Entry Time', 'Price', 'Status', 'Profit/Loss Amount'])

# Save the results to an Excel file
output_file_path = 'C:\\Users\\omkar\\Downloads\\stock_results.xlsx'
results_df.to_excel(output_file_path, index=False)

print(f"Results saved to {output_file_path}")
print(f"Total results: {len(results)}")